If the process crashes while we are waiting for Anthropic batch jobs to complete, this code can pull the results down when they are ready and put them in the cache

In [1]:
from repsheet_backend.genai import anthropic, genai_cache
import asyncio

In [ ]:
SAVED_BATCH_IDS_FILE = "repsheet_backend/data/saved_batch_ids.txt"

with open(SAVED_BATCH_IDS_FILE, "r") as f:
    saved_batch_ids = f.read().splitlines()

async def save_batch_to_cache(batch_id: str):
    if batch_id in saved_batch_ids:
        return
    results = await asyncio.to_thread(anthropic.messages.batches.results, batch_id)
    saved_count = 0
    cache_set_jobs = []
    for result in results:
        cache_key = result.custom_id
        # there are some old jobs with numeric custom_ids
        if len(cache_key) > 10:
            cache_set_jobs.append(genai_cache.set(cache_key, result.result.message.content[0].text))
            saved_count += 1
    await asyncio.gather(*cache_set_jobs)
    print(f"Saved {saved_count} results for {batch_id}")
    saved_batch_ids.append(batch_id)

In [ ]:
await asyncio.gather(*[
    save_batch_to_cache(message.id)
    for message in anthropic.messages.batches.list()
    if message.processing_status == "ended"
])

for message in anthropic.messages.batches.list():
    if message.processing_status != "ended":
        print(f"Batch {message.id} is still {message.processing_status}")



Batch msgbatch_01BVGT8PrSECWc5mEuDMcm2q already saved
Batch msgbatch_012E6aERzmoStuit7nghobCC already saved
Batch msgbatch_011UWoaFT4d32ykixGqaKBho already saved
Batch msgbatch_015JruEfXYS443wjtvR44Tfu already saved
Batch msgbatch_01WK2zAYu47yXdB2H2cvD4AG already saved
Batch msgbatch_014aMUr8Cv46fM7wbrNh6mso already saved
Batch msgbatch_013JXaq6xZi46HqyaS7V4hK3 already saved
Batch msgbatch_01Da9XjUgLYKyPo4RwJNAZNA already saved
Batch msgbatch_01HnLJpT89RGQqvNDUWGDs1x already saved
Batch msgbatch_01N51foigwrjtusn7dEmExXj already saved
Batch msgbatch_016xmH41PvqfmjSiQNbkbvoK already saved
Batch msgbatch_019U74kaM9jkwsjtjmfc6HBD already saved
Batch msgbatch_01QP4h2McJfqZSLN18w2dVVn already saved
Batch msgbatch_016W8Wi8NibpqcbXA19nU8xm already saved
Batch msgbatch_01VMHWkquLdTJPZFDGWtNcRi already saved
Batch msgbatch_01RJBNEtrDFvVuC2XAvEWjhK already saved
Batch msgbatch_01EVrVnt6urEGgTtodkWVy2j already saved
Batch msgbatch_0129eqQX4ytYrPnqVcnnsWwa already saved
Batch msgbatch_01GnxCDd5dxZf

In [ ]:
with open(SAVED_BATCH_IDS_FILE, "w") as f:
    f.write("\n".join(saved_batch_ids))